In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tabulate import tabulate

In [2]:
# производная по центральной разностной схеме
def diff(func, x_0, h=10**-5):
    if np.linalg.norm(h) < 10**-12:
        h = 10**-12 * np.linalg.norm(h)
    return (func(x_0 + h) - func(x_0 - h)) / np.linalg.norm(2 * h)

# численный градиент
def grad(func, x_0, h = 10**-5):
    dim = len(x_0)
    counter = 0

    gradient_vector = np.array([])
    for i in range(dim):
        vec = np.array([0] * dim, dtype=float)
        vec[i] += h
        gradient_vector = np.append(gradient_vector, diff(func, x_0, vec))
        counter += 2

    return gradient_vector, counter

# численная матрица Гессе
def hesse(func, x_0, grad_func=None, h=10**-5):
    dim = len(x_0)
    hesse_matrix = np.zeros((dim, dim))

    if grad_func is None:
        f_at_x0 = func(x_0)
        counter = 1
        
        for i in range(dim):
            vec = np.zeros(dim)
            vec[i] = h
            hesse_matrix[i, i] = (func(x_0 + vec) - 2 * f_at_x0 + func(x_0 - vec)) / h**2
            counter += 2
            for j in range(i + 1, dim):
                vec_2 = np.zeros(dim)
                vec_2[j] = h
                hesse_matrix[i, j] = (func(x_0 + vec + vec_2) - func(x_0 - vec + vec_2) - func(x_0 + vec - vec_2) + func(x_0 - vec - vec_2)) / (4 * h**2)
                counter += 4
                hesse_matrix[j, i] = hesse_matrix[i, j].copy()
    else:
        for i in range(dim):
            vec = np.zeros(dim)
            vec[i] = h
            hesse_matrix[1, :] = (grad_func(x_0 + vec) - grad_func(x_0 - vec)) / (2 * h)
            counter += 2 * dim

    return hesse_matrix, counter

# численная матрица Гессе + халявный (относительно вызовов функции func) градиент
def hesse_and_grad(func, x_0, h=10**-5):
    dim = len(x_0)
    hesse_matrix = np.zeros((dim, dim))
    gradient = np.zeros(dim)
    f_at_x0 = func(x_0)
    counter = 1
    
    for i in range(dim):
        vec = np.zeros(dim)
        vec[i] = h
        f_plus = func(x_0 + vec)
        f_minus = func(x_0 - vec)
        hesse_matrix[i, i] = (f_plus - 2 * f_at_x0 + f_minus) / h**2
        gradient[i] = (f_plus - f_minus) / (2 * h)
        counter += 2
        
        for j in range(i + 1, dim):
            vec_2 = np.zeros(dim)
            vec_2[j] = h
            hesse_matrix[i, j] = (func(x_0 + vec + vec_2) - func(x_0 - vec + vec_2) - func(x_0 + vec - vec_2) + func(x_0 - vec - vec_2)) / (4 * h**2)
            counter += 4
            hesse_matrix[j, i] = hesse_matrix[i, j].copy()

    return hesse_matrix, gradient, counter

# метод поразрядного поиска
def bitwise_search(f, x_0, vec, alpha=1, eps=10**-8):
    x = x_0
    f_val = f(x_0)
    delta = alpha * vec / np.linalg.norm(vec)
    counter = 1

    while True:
        while True:
            f_min = f_val
            x += delta
            f_val = f(x)
            counter += 1
            if f_val > f_min:
                break
                
        if np.linalg.norm(delta) < eps:
            x_min = x - delta
            break
    
        delta /= -4
        
    return x_min, f_val, counter






# метод наискорейшего спуска
def steepest_descent(f, x_0, grad_func=None, bitwise_eps=10**-8, eps=10**-5, max_iter=100, exact_sol=None):
    dim = len(x_0)
        
    x_min = x_0
    f_min = f(x_0)
    counter = 1
    iterations = 0
    
    while iterations < max_iter:
        iterations += 1

        if grad_func is None:
            gradient, tmp_counter = grad(f, x_min)
            counter += tmp_counter
        else:
            gradient = grad_func(x_min)
            counter += dim

        gradient_norm = np.linalg.norm(gradient)
        if not exact_sol is None:
            if np.linalg.norm(np.array(exact_sol) - x_min) < eps:
                break
        elif gradient_norm < eps or (iterations > 1 and abs(gradient_norm - last_gradient_norm) < eps):
            break
        last_gradient_norm = gradient_norm

        x_min, f_min, tmp_counter = bitwise_search(f, x_min, -gradient, eps=bitwise_eps)
        counter += tmp_counter

    return x_min, f_min, counter, iterations

# метод сопряженных градиентов
def conjugate_gradient(f, x_0, N=None, grad_func=None, bitwise_eps=10**-8, eps=10**-5, max_iter=100, exact_sol=None):
    dim = len(x_0)
    if N is None: N = dim
        
    x_min = x_0
    if grad_func is None:
        gradient, counter = grad(f, x_min)
    else:
        gradient = grad_func(x_min)
        counter = dim
    last_gradient_norm = np.linalg.norm(gradient)
    vec = -gradient
    beta = 0
    iterations = 0
    
    while iterations < max_iter:
        iterations += 1
        
        x_min, f_min, tmp_counter = bitwise_search(f, x_min, vec, eps=bitwise_eps)
        if grad_func is None:
            gradient, tmp_counter = grad(f, x_min)
            counter += tmp_counter
        else:
            gradient = grad_func(x_min)
            counter += dim
        gradient_norm = np.linalg.norm(gradient)

        if not exact_sol is None:
            if np.linalg.norm(np.array(exact_sol) - x_min) < eps:
                break
        elif gradient_norm < eps or abs(gradient_norm - last_gradient_norm) < eps:
            break
        
        if iterations % N:
            beta = (gradient_norm / last_gradient_norm) ** 2
        else:
            beta = 0
            
        last_gradient_norm = gradient_norm
        vec = -gradient + beta * vec

    return x_min, f_min, counter, iterations
        
# метод Ньютона
def newton(f, x_0, grad_func=None, hesse_func=None, eps=10**-5, max_iter=100, exact_sol=None):
    dim = len(x_0)
    
    x_min = x_0
    counter = 0
    iterations = 0
    
    while iterations < max_iter:
        iterations += 1

        if hesse_func is None and grad_func is None:
            hesse_matrix, gradient, tmp_counter = hesse_and_grad(f, x_min)
            counter += tmp_counter
        elif hesse_func is None and not grad_func is None:
            hesse_matrix, tmp_counter = hesse(f, x_min, grad_func=grad_func)
            gradient = grad_func(x_min)
            counter += tmp_counter + dim
        elif not hesse_func is None and grad_func is None:
            hesse_matrix = hesse_func(x_min)
            gradient, tmp_counter = grad(f, x_min)
            counter += dim * (dim - 1) / 2 + tmp_counter
        else:
            hesse_matrix = hesse_func(x_min)
            gradient = grad_func(x_min)
            counter += dim * (dim - 1) / 2 + dim

        if not exact_sol is None:
            if np.linalg.norm(np.array(exact_sol) - x_min) < eps:
                break
        elif np.linalg.norm(gradient) < eps:
            break
        
        x_min -= (np.linalg.inv(hesse_matrix) @ gradient.T).T

    return x_min, f(x_min), counter, iterations

# метод правильного симплекса
def simplex(f, x_0, l=2, delta=0.5, eps=10**-5, max_iter=100, exact_sol=None):
    dim = len(x_0)
    
    template = np.zeros((dim + 1, dim))
    template[0, :] = x_0
    for i in range(1, dim + 1):
        vertex = np.zeros(dim)
        for j in range(dim):
            if i == j + 1: vertex[j] = x_0[j] + (np.sqrt(dim + 1) - 1) / (dim * np.sqrt(2)) * l
            else: vertex[j] = x_0[j] + (np.sqrt(dim + 1) + dim - 1) / (dim * np.sqrt(2)) * l
        template[i, :] = vertex
        
    template_f = f(template.T)
    sorting = template_f.argsort()
    template = template[sorting]
    template_f = template_f[sorting]
    counter = dim + 1
    iterations = 0
    
    while iterations < max_iter:
        if not exact_sol is None:
            if np.linalg.norm(np.array(exact_sol) - template[0]) < eps:
                break
        else:
            if l >= eps:
                break  
            counter += 1
            if np.sqrt(sum((template_f - f(sum(template) / (dim + 1)))**2) / (dim + 1)) < eps:
                break
        iterations += 1
        
        new_x = 2 / dim * sum(template[:-1]) - template[-1]
        new_f = f(new_x)
        counter += 1
        
        if new_f > template_f[-1]:
            l *= delta
            for i in range(1, dim + 1):
                template[i] = template[0] + delta * (template[i] - template[0])
            template_f[1:] = f(template[1:].T)
            counter += dim
        else:
            template[-1] = new_x
            template_f[-1] = new_f

        sorting = template_f.argsort()
        template = template[sorting]
        template_f = template_f[sorting]
        
    return template[0], template_f[0], counter, iterations

# метод циклического покоординатного спуска
def cyclic_descent(f, x_0, bitwise_eps=10**-8, eps=10**-5, max_iter=100, exact_sol=None):
    dim = len(x_0)
        
    x_min = x_0
    f_min = f(x_0)
    counter = 1
    iterations = 0

    while iterations < max_iter:
        iterations += 1
        last_x = x_min
        last_f = f_min
        
        for i in range(dim):
            vec = np.zeros(dim)
            vec[i] = 1
            x_min, f_min, tmp_counter = bitwise_search(f, x_min, vec, eps=bitwise_eps)
            counter += tmp_counter

        if not exact_sol is None:
            if np.linalg.norm(np.array(exact_sol) - x_min) < eps:
                break
        elif abs(last_f - f_min) < eps or np.sqrt(sum((last_x - x_min)**2)) < eps:
            break

    return x_min, f_min, counter, iterations

# метод Хука-Дживса
def hooke_jeeves(f, x_0, delta=None, gamma=2, bitwise=False, bitwise_eps=10**-8, alpha=1, eps=10**-5, max_iter=100, exact_sol=None):
    dim = len(x_0)
        
    x_min = np.array(x_0, dtype=np.float64)
    f_min = f(x_0)
    counter = 1
    iterations = 0

    if delta is None:
        delta = np.ones(dim)

    while iterations < max_iter:
        iterations += 1
        last_x = x_min.copy()
        last_f = f_min

        # исследующий поиск
        for i in range(dim):
            vec = np.zeros(dim)
            vec[i] = delta[i]
            new_f = f(x_min - vec)
            counter += 1
            if new_f < f_min:
                x_min -= vec
                f_min = new_f
                continue
                
            new_f = f(x_min + vec)
            counter += 1
            if new_f < f_min:
                x_min += vec
                f_min = new_f

        # перемещение
        if sum(np.abs(last_x - x_min)) < eps:
            if not exact_sol is None:
                if np.linalg.norm(np.array(exact_sol) - x_min) < eps:
                    break
            elif np.linalg.norm(delta) < eps: 
                break
            delta /= gamma
        else:
            if bitwise:
                x_min, f_min, tmp_counter = bitwise_search(f, x_min, x_min - last_x, eps=bitwise_eps)
                counter += tmp_counter
            elif alpha != 1:
                x_min = last_x + alpha * (x_min - last_x)
                f_min = f(x_min)
                counter += 1

    return x_min, f_min, counter, iterations

# метод случайного поиска
def random_search(f, x_0, alpha=1, gamma=2, M=80, eps=10**-5, max_iter=100, exact_sol=None):
    dim = len(x_0)
    
    x_min = x_0
    f_min = f(x_0)
    counter = 1
    iterations = 0

    while iterations < max_iter:
        iterations += 1
        
        ksi = np.random.uniform(-1, 1, dim)
        for j in range(M):
            x_new = x_min + alpha * ksi / np.linalg.norm(ksi)
            f_new = f(x_new)
            counter += 1

            if f_min < f_new: ksi = np.random.uniform(-1, 1, dim)
            else:
                x_min = x_new
                f_min = f_new
                j -= 1

        if not exact_sol is None:
            if np.linalg.norm(np.array(exact_sol) - x_min) < eps:
                break
        elif alpha < eps: 
            break
        alpha /= gamma

    return x_min, f_min, counter, iterations

In [3]:
function = lambda x: x[0]**2 + a * x[1]**2
parameters = [1, 250, 1000]

digits = 6
tolerances = [10**(-i) for i in range(1, digits + 1)]
#tolerances = [10**-3, 10**-5]

method_names = ["Метод наискорейшего спуска", "Метод сопряженных градиентов", "Метод Ньютона", "Метод правильного симплекса", "Метод циклического покоординатного спуска", 
                "Метод Хука-Дживса", "Метод случайного поиска"]
methods = [steepest_descent, conjugate_gradient, newton, simplex, cyclic_descent, hooke_jeeves, random_search]
starting_points = [(-10.6737344875, 10.75586373563)]
#starting_points = [(10.232342, 2.97896578), (-1.123531, -2.3474368)]

for x_0 in starting_points:
    for a in parameters:
        for i in range(len(method_names)):
            print(f"{method_names[i]} (x_0 = {x_0}, a = {a})")
            table = []
            for eps in tolerances:
                x_min, f_min, f_calls, iterations = methods[i](function, x_0, eps=eps, max_iter=50000)
                x_min = tuple([f"{{:.{digits}f}}".format(x) for x in x_min])
                table.append([eps, x_min, f_min, f_calls, iterations])
            print(tabulate(table, headers=["Precision", "x_min", "f_min", "f calls", "Iterations"], tablefmt='orgtbl', floatfmt=f".{digits}f"))
            print()
            
        print("\n\n\n\n")

Метод наискорейшего спуска (x_0 = (-10.6737344875, 10.75586373563), a = 1)
|   Precision | x_min                      |    f_min |   f calls |   Iterations |
|-------------+----------------------------+----------+-----------+--------------|
|    0.100000 | ('-0.000000', '-0.000000') | 0.000000 |        86 |            2 |
|    0.010000 | ('-0.000000', '-0.000000') | 0.000000 |        86 |            2 |
|    0.001000 | ('-0.000000', '-0.000000') | 0.000000 |        86 |            2 |
|    0.000100 | ('-0.000000', '-0.000000') | 0.000000 |        86 |            2 |
|    0.000010 | ('-0.000000', '-0.000000') | 0.000000 |        86 |            2 |
|    0.000001 | ('-0.000000', '-0.000000') | 0.000000 |        86 |            2 |

Метод сопряженных градиентов (x_0 = (-10.6737344875, 10.75586373563), a = 1)
|   Precision | x_min                      |    f_min |   f calls |   Iterations |
|-------------+----------------------------+----------+-----------+--------------|
|    0.100000 | (

In [4]:
a = 250
functions = [function, lambda x: 85 * x[0]**2 + 168 * x[0] * x[1] + 85 * x[1]**2 + 29 * x[0] - 51 * x[1] + 83]
function_names = ["функция из п.2", "функция из п.3"]

digits = 6
tolerances = [10**(-i) for i in range(1, digits + 1)]
#tolerances = [10**-3, 10**-5]

method_names = ["Метод наискорейшего спуска", "Метод сопряженных градиентов", "Метод Ньютона", "Метод правильного симплекса", "Метод циклического покоординатного спуска", 
                "Метод Хука-Дживса", "Метод случайного поиска"]
methods = [steepest_descent, conjugate_gradient, newton, simplex, cyclic_descent, hooke_jeeves, random_search]
starting_points = [(-10.6737344875, 10.75586373563)]
#starting_points = [(-18.1342, 18.97896578), (1.123531, -2.3474368)]

for x_0 in starting_points:
    for i in range(len(method_names)):
        for j in range(len(function_names)):
            print(f"{method_names[i]} ({function_names[j]}, x_0 = {x_0})")
            table = []
            for eps in tolerances:
                x_min, f_min, f_calls, iterations = methods[i](functions[j], x_0, eps=eps, max_iter=50000)
                x_min = tuple([f"{{:.{digits}f}}".format(x) for x in x_min])
                table.append([eps, x_min, f_min, f_calls, iterations])
            print(tabulate(table, headers=["Precision", "x_min", "f_min", "f calls", "Iterations"], tablefmt='orgtbl', floatfmt=f".{digits}f"))
            print()
            
        print("\n\n")
    print("\n\n\n")

Метод наискорейшего спуска (функция из п.2, x_0 = (-10.6737344875, 10.75586373563))
|   Precision | x_min                      |    f_min |   f calls |   Iterations |
|-------------+----------------------------+----------+-----------+--------------|
|    0.100000 | ('-0.041375', '-0.000001') | 0.001712 |       256 |            4 |
|    0.010000 | ('-0.000161', '0.000163')  | 0.000007 |       339 |            5 |
|    0.001000 | ('-0.000161', '-0.000000') | 0.000000 |       417 |            6 |
|    0.000100 | ('-0.000000', '0.000001')  | 0.000000 |       495 |            7 |
|    0.000010 | ('-0.000000', '-0.000000') | 0.000000 |       579 |            8 |
|    0.000001 | ('-0.000000', '0.000000')  | 0.000000 |       656 |            9 |

Метод наискорейшего спуска (функция из п.3, x_0 = (-10.6737344875, 10.75586373563))
|   Precision | x_min                       |       f_min |   f calls |   Iterations |
|-------------+-----------------------------+-------------+-----------+---------

In [5]:
rosenbrock = lambda x: 100 * (x[0]**2 - x[1])**2 + (x[0] - 1)**2

digits = 6
tolerances = [10**(-i) for i in range(1, digits + 1)]
bitwise_tolerances = [10**-4, 10**-6, 10**-8]
#tolerances = [10**-3, 10**-5]

method_names = ["Метод наискорейшего спуска", "Метод сопряженных градиентов", "Метод Ньютона", "Метод правильного симплекса", "Метод циклического покоординатного спуска", 
                "Метод Хука-Дживса", "Метод случайного поиска"]
methods = [steepest_descent, conjugate_gradient, newton, simplex, cyclic_descent, hooke_jeeves, random_search]
starting_points = [(-1, 1)]
solution = (1, 1)

for x_0 in starting_points:
    for i in range(len(method_names)):
        if not method_names[i] in ["Метод наискорейшего спуска", "Метод сопряженных градиентов", "Метод циклического покоординатного спуска"]:  
            print(f"{method_names[i]} (x_0 = {x_0})")
            table = []
            for eps in tolerances:
                x_min, f_min, f_calls, iterations = methods[i](rosenbrock, x_0, eps=eps, max_iter=50000, exact_sol=solution)
                x_min = tuple([f"{{:.{digits}f}}".format(x) for x in x_min])
                table.append([eps, x_min, f_min, f_calls, iterations])
            print(tabulate(table, headers=["Precision", "x_min", "f_min", "f calls", "Iterations"], tablefmt='orgtbl', floatfmt=f".{digits}f"))
        else:
            print()
            for bit_eps in bitwise_tolerances:
                print(f"{method_names[i]} (x_0 = {x_0}, bit_eps = {bit_eps})")
                table = []
                for eps in tolerances:
                    x_min, f_min, f_calls, iterations = methods[i](rosenbrock, x_0, bitwise_eps=bit_eps, eps=eps, max_iter=50000, exact_sol=solution)
                    x_min = tuple([f"{{:.{digits}f}}".format(x) for x in x_min])
                    table.append([eps, x_min, f_min, f_calls, iterations])
                print(tabulate(table, headers=["Precision", "x_min", "f_min", "f calls", "Iterations"], tablefmt='orgtbl', floatfmt=f".{digits}f"))
                print()
        print()
        
    print("\n\n\n")


Метод наискорейшего спуска (x_0 = (-1, 1), bit_eps = 0.0001)
|   Precision | x_min                    |    f_min |   f calls |   Iterations |
|-------------+--------------------------+----------+-----------+--------------|
|    0.100000 | ('0.973340', '0.948334') | 0.000800 |     24002 |          563 |
|    0.010000 | ('0.995501', '0.991072') | 0.000021 |    108883 |         2685 |
|    0.001000 | ('0.999527', '0.999120') | 0.000001 |    223843 |         5559 |
|    0.000100 | ('0.999863', '0.999658') | 0.000004 |   2044706 |        50000 |
|    0.000010 | ('0.999863', '0.999658') | 0.000004 |   2044706 |        50000 |
|    0.000001 | ('0.999863', '0.999658') | 0.000004 |   2044706 |        50000 |

Метод наискорейшего спуска (x_0 = (-1, 1), bit_eps = 1e-06)
|   Precision | x_min                    |    f_min |   f calls |   Iterations |
|-------------+--------------------------+----------+-----------+--------------|
|    0.100000 | ('0.954475', '0.911011') | 0.002073 |     64483 |  

In [6]:
himmelblau = lambda x: (x[0]**2 + x[1] - 11)**2 + (x[0] + x[1]**2 - 7)**2

digits = 6
tolerances = [10**(-i) for i in range(1, digits + 1)]
#tolerances = [10**-3, 10**-5]

method_names = ["Метод наискорейшего спуска", "Метод сопряженных градиентов", "Метод Ньютона"]
methods = [steepest_descent, conjugate_gradient, newton]
starting_points = [(0, 0), (-5, 0), (2, 1)]

for x_0 in starting_points:
    for i in range(len(method_names)):
        print(f"{method_names[i]} (x_0 = {x_0})")
        table = []
        for eps in tolerances:
            x_min, f_min, f_calls, iterations = methods[i](himmelblau, x_0, eps=eps, max_iter=10000)
            x_min = tuple([f"{{:.{digits}f}}".format(x) for x in x_min])
            table.append([eps, x_min, f_min, f_calls, iterations])
        print(tabulate(table, headers=["Precision", "x_min", "f_min", "f calls", "Iterations"], tablefmt='orgtbl', floatfmt=f".{digits}f"))
        print()
            
    print("\n\n\n\n")

Метод наискорейшего спуска (x_0 = (0, 0))
|   Precision | x_min                    |    f_min |   f calls |   Iterations |
|-------------+--------------------------+----------+-----------+--------------|
|    0.100000 | ('3.000043', '2.001600') | 0.000045 |       456 |            7 |
|    0.010000 | ('2.999873', '2.000183') | 0.000001 |       678 |           10 |
|    0.001000 | ('3.000001', '2.000025') | 0.000000 |       918 |           13 |
|    0.000100 | ('3.000000', '2.000002') | 0.000000 |      1224 |           17 |
|    0.000010 | ('3.000000', '2.000000') | 0.000000 |      1450 |           20 |
|    0.000001 | ('3.000000', '2.000000') | 0.000000 |      1681 |           23 |

Метод сопряженных градиентов (x_0 = (0, 0))
|   Precision | x_min                    |    f_min |   f calls |   Iterations |
|-------------+--------------------------+----------+-----------+--------------|
|    0.100000 | ('3.000127', '2.001274') | 0.000031 |        28 |            6 |
|    0.010000 | ('3.00